In [1]:
from github import Github
from string import Template

In [4]:
g = Github("oauth")
org = g.get_organization("jay-feng-org")

# Import an Existing Repository


This is a little more complicated than the other two: when we import a repository, we also need to import and create resource blocks for each of the teams and individual users with access permissions for that repository.

In [5]:
repo = org.get_repo("team_1_test_repo")

In [7]:
# good news! terraform has the command `terraform fmt` which automatically formats 
# the code, so we can write ugly long dictionaries (maps) like this one below


"""
Takes in a PyGithub Repository object, returns a string of an HCL map representing the individual
user permissions of a repository.
"""
def generate_repo_user_permissions(repo):
    user_dict = {}
    for c in repo.get_collaborators():
        if repo.get_collaborator_permission(c) != "read":
            user_dict[c.login] =  repo.get_collaborator_permission(c)
        
    user_dict_string = str(user_dict)
    user_dict_string = user_dict_string.replace("\'", "\"").replace(":", " =").replace(",", "\n")
    user_dict_string = user_dict_string.replace("{", "{\n ").replace("}", "\n}")
    # the below makes indents, which is not necessary with `terraform fmt`
    # user_dict_string = user_dict_string.replace("\n ", "\n  ")
    
    return user_dict_string
        
        
#print(generate_repo_user_permissions(repo))

In [8]:
"""
Helper function, takes in a PyGithub Permissions object, returns the Terraform permission value.
Raises a ValueError if the permission object does not grant any permissions, or if the permission is only 'read'.
"""
def get_permission(permissions):
    if permissions.triage:
        return "triage"
    elif permissions.push:
        return "push"
    elif permissions.pull:
        return "pull"
    elif permissions.maintain:
        return "maintain"
    elif permissions.admin:
        return "admin"
    else:
        raise ValueError('Team does not have any permissions.')

In [9]:
"""
Takes in a PyGithub Repository object, returns a string of an HCL map representing the team
permissions of a repository.
"""
def generate_repo_team_permissions(repo):
    team_dict = {}
    for t in repo.get_teams():
        team_dict[t.name] =  get_permission(t.get_repo_permission(repo))
        
    team_dict_string = str(team_dict)
    team_dict_string = team_dict_string.replace("\'", "\"").replace(":", " =").replace(",", "\n")
    team_dict_string = team_dict_string.replace("{", "{\n ").replace("}", "\n}")
    # the below makes indents, which is not necessary with `terraform fmt`
    # team_dict_string = team_dict_string.replace("\n ", "\n  ")
    return team_dict_string
        
        
#print(generate_repo_team_permissions(repo))

In [10]:
"""
Takes in a repository name, returns an string of the HCL repository module configuration.
"""
def generate_repo_config(repo_name):
    repo = org.get_repo(repo_name)
 
    module_name = "make_" + repo_name
    description = "null" if (repo.description is None) else repo.description
    visibility = "private" if repo.private else "public"
    allow_merge_commit = "true" if repo.allow_merge_commit else "false"
    
    users = generate_repo_user_permissions(repo)
    teams = generate_repo_team_permissions(repo)
    
    # store the template string in a file and read it in
    t = Template('module "$module_name" {\n  source = "./modules/repo"\n\n  token = var.token\n\n  repo_name   = "$repo_name"\n  description = "$description"\n  visibility = "$visibility"\n  allow_merge_commit = "$allow_merge_commit"\n\n  users = $users\n\n  teams = $teams\n}')
    return t.substitute(module_name = module_name,
                        repo_name = repo_name,
                        description = description,
                        visibility = visibility,
                        allow_merge_commit = allow_merge_commit,
                        users = users,
                        teams = teams)

                    
print(generate_repo_config("team_1_test_repo"))

module "make_team_1_test_repo" {
  source = "./modules/repo"

  token = var.token

  repo_name   = "team_1_test_repo"
  description = "null"
  visibility = "public"
  allow_merge_commit = "true"

  users = {
 "jay-feng-ge" = "admin"
}

  teams = {
 "team_1" = "pull"
}
}


### Directions
1. run `terraform fmt` to clean up and add indents
2. copy the generated organization membership config block into a .tf file (teams.tf or imported_teams.tf)
3. run `terraform init` to make sure the resource names exist in the terraform state
4. run `terraform import module.make_team_1_test_repo.github_repository.some_repo team_1_test_repo`

### Things to add to repositories
- branch protections *
- access *
- security analysis *
- webhooks
- notifications
- integrations
- deploy keys *
- actions (ignore for now)
- secrets (skip for now) *

skel system - templates to automatically configure (ex. gitignore)
- ex. set of github actions preconfigured to be copied into new repos created through terraform